<a href="https://colab.research.google.com/github/KaranTejwani/deep-learning-practise/blob/main/Sentiment_analysis_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_path = '/content/drive/My Drive/datasets/sentiment140.csv'

In [3]:
import pandas as pd

df = pd.read_csv(dataset_path, encoding='latin-1', header=None)
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
display(df.head())

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
dataset = df[['sentiment', 'text']]

In [5]:
dataset

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and convert to lowercase
    cleaned_text = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(cleaned_text)

In [8]:
dataset['text'] = dataset['text'].apply(clean_text)
display(dataset.head())

/tmp/ipython-input-3349414461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(clean_text)


,sentiment,text
0,0,switchfoot httptwitpiccom2y1zl awww thats bumm...
1,0,upset cant update facebook texting might cry r...
2,0,kenichan dived many times ball managed save 50...
3,0,whole body feels itchy like fire
4,0,nationwideclass behaving im mad cant see


In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataset, test_size=0.3, random_state=42, stratify=dataset['sentiment'])

In [10]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['sentiment'])

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=10000,
    output_mode='multi_hot'
)

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['text'], train_df['sentiment']))
val_ds   = tf.data.Dataset.from_tensor_slices((val_df['text'], val_df['sentiment']))
test_ds  = tf.data.Dataset.from_tensor_slices((test_df['text'], test_df['sentiment']))

In [13]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [14]:
text_vectorization.adapt(text_only_train_ds)

train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
val_ds   = val_ds.map(lambda x, y: (text_vectorization(x), y))
test_ds  = test_ds.map(lambda x, y: (text_vectorization(x), y))

In [17]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs:", inputs)
    print("targets:", targets)
    break

inputs.shape: (10000,)
inputs.dtype: <dtype: 'int64'>
targets.shape: ()
targets.dtype: <dtype: 'int64'>
inputs: tf.Tensor([1 0 0 ... 0 0 0], shape=(10000,), dtype=int64)
targets: tf.Tensor(4, shape=(), dtype=int64)
